In [1]:
import librosa
import numpy as np
import soundfile as sf
import warnings

warnings.filterwarnings("ignore")

In [2]:
def enhance_audio(audio_file, beats_gain=0.0):

    try:
        # 1. Загрузка аудио с помощью Librosa
        y, sr = librosa.load(audio_file)

        # 2. Определение битов
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        beat_times = librosa.frames_to_time(beat_frames, sr=sr)

        # 3. Создание маски для битов
        beat_mask = np.zeros_like(y)
        beat_duration = np.mean(np.diff(beat_times)) / 2  # Половина расстояния между битами
        for beat_time in beat_times:
            start_frame = librosa.time_to_samples(beat_time - beat_duration, sr=sr)
            end_frame = librosa.time_to_samples(beat_time + beat_duration, sr=sr)
            start_frame = max(0, start_frame)
            end_frame = min(len(y), end_frame)
            beat_mask[start_frame:end_frame] = 1

        # 4. Применение усиления
        y_enhanced = y.copy()

        # Усиление битов
        y_enhanced[beat_mask == 1] *= (10**(beats_gain/20))  # Преобразование дБ в коэффициент усиления

        # 5. Нормализация
        y_enhanced = np.clip(y_enhanced, -1, 1)

        # 6. Запись измененного аудио
        output_file = "enhanced_audio.wav"
        sf.write(output_file, y_enhanced, sr)

        return output_file

    except Exception as e:
        print(f"Ошибка при обработке аудио: {e}")
        return None

In [4]:
audio_file = "Beethoven_Violin_Sonata_Op_96_first_movement_bars_1-22.wav"
enhanced_file = enhance_audio(audio_file, beats_gain=6.0)

if enhanced_file:
    print(f"Аудио обработано и сохранено как: {enhanced_file}")
else:
    print("Обработка аудио не удалась.")

Аудио обработано и сохранено как: enhanced_audio.wav
